Testing derivaties for 1D MT problem.

Especially the rx.projectFieldsDeriv

In [1]:
import SimPEG as simpeg
import simpegEM as simpegem, simpegMT as simpegmt
from SimPEG.Utils import meshTensor
import numpy as np

In [2]:
simpegmt.FieldsMT.FieldsMT_1D

simpegMT.FieldsMT.FieldsMT_1D

In [3]:
# Setup the problem
sigmaHalf = 1e-2
# Frequency
nFreq = 33
# freqs = np.logspace(3,-3,nFreq)
freqs = np.array([100])
# Make the mesh
ct = 5
air = meshTensor([(ct,25,1.3)])
# coreT0 = meshTensor([(ct,15,1.2)])
# coreT1 = np.kron(meshTensor([(coreT0[-1],15,1.3)]),np.ones((7,)))
core = np.concatenate( (  np.kron(meshTensor([(ct,15,-1.2)]),np.ones((10,))) , meshTensor([(ct,20)]) ) )
bot = meshTensor([(core[0],10,-1.3)])
x0 = -np.array([np.sum(np.concatenate((core,bot)))])
# Change to use no air
m1d = simpeg.Mesh.TensorMesh([np.concatenate((bot,core))], x0=x0)
# Make the model
sigma = np.zeros(m1d.nC) + sigmaHalf
sigma[ m1d.gridCC > 0 ] = 1e-8

rxList = []
for rxType in ['z1dr','z1di']:
    rxList.append(simpegmt.SurveyMT.RxMT(simpeg.mkvc(np.array([0.0]),2).T,rxType))
# Source list
srcList =[]
tD = False
if tD:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1DhomotD(rxList,freq))
else:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1Dprimary(rxList,freq,sigma))
# Make the survey
survey = simpegmt.SurveyMT.SurveyMT(srcList)

# Set the problem
problem = simpegmt.ProblemMT1D.eForm_psField(m1d)
problem.pair(survey)

# Get the fields
fields = problem.fields(sigma)

# Project the data
data = survey.projectFields(fields)


Project at freq: 1.000e+02


We need calculate this derivative. 
\begin{align}
\underbrace{\frac{\partial P(f(u(m)),m^{fix})}{\partial f}}_{Rx}
\end{align}

Use the rule
\begin{align}
\frac{d}{dx}\left( \frac{a(x)}{b(x)} \right)  = \frac{\frac{d }{dx} a(x)  b(x) - a(x)\frac{d }{dx} b(x)  }{ b(x)^2 }
\end{align}

In the case of the 1D MT problem the data is calculated as 
\begin{align}
MT1Ddata = P(f(m)) &= \frac{P_{ex} f_e(src,m)}{P_{bx} f_b(src,m) \frac{1}{\mu_0}} = \frac{P_e u}{P_b f_b(u)} \\
\frac{\partial P(f(m))}{\partial u} v &=  \frac{P_e}{P_b \frac{1}{mu_0} f_b(u)}v - \frac{P_e u}{\left(P_b \frac{1}{mu_0} f_b(u)\right)^2} P_b \frac{1}{mu_0} \frac{d f_b}{du} v
\end{align}
where u is the fields that we solve for. 
\begin{align}
\frac{d f_b}{du} = - \frac{1}{i \omega} \nabla 
\end{align}



In [4]:
# Unused code &= \frac{ P_{ex} P_{bx} \frac{1}{\mu_0} \left( f_b(src,m) - f_e(src,m) \right) } { \left(P_{bx}f_b(src,m) \frac{1}{\mu_0} \right)^2 }

As matrices the formulas above can be written as
\begin{align}
\left[ \frac{\partial P(f(m))}{\partial u} v \right] = diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] [P_e v] - diag[P_e u] diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] \left[ P_b   \frac{d f_b}{du}(v) \frac{1}{mu_0} \right]
\end{align}



The adjoint problem is done simliarly
\begin{align}
\left[ \frac{\partial P(f(m))}{\partial u} v \right]^T = [P_e v]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T  - \left[ P_b   \frac{d f_b}{du}(v) \frac{1}{mu_0} \right]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T diag \left[ P_e u  \right]^T
\end{align}


In [5]:
# def projectFields(self, src, mesh, u):
#         '''
#         Project the fields and return the
#         '''

#         if self.projType is 'Z1D':
#             Pex = mesh.getInterpolationMat(self.locs,'Fx')
#             Pbx = mesh.getInterpolationMat(self.locs,'Ex')
#             ex = Pex*mkvc(u[src,'e_1d'],2)
#             bx = Pbx*mkvc(u[src,'b_1d'],2)/mu_0
#             f_part_complex = ex/bx
#         real_or_imag = self.projComp
#         f_part = getattr(f_part_complex, real_or_imag)
#         return f_part

In [6]:
# Initate things for the derivs Test
src = survey.srcList[0]
rx = src.rxList[0]
v = np.random.randn(m1d.nN)
u0 = np.random.randn(m1d.nN)+np.random.randn(m1d.nN)*1j
f0 = problem.fieldsPair(m1d,survey)
f0[src,'e_1dSolution'] = u0
# f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0

In [7]:
u0

array([ -5.21576299e-01-0.28596337j,   1.33775588e+00-1.08159078j,
         1.02311300e+00-0.45033307j,  -1.14140017e+00-0.53814712j,
        -1.51095317e+00-0.0225078j ,   2.29842631e-01-0.33435588j,
        -6.13742564e-01+0.14777572j,   1.74709600e+00+0.07973247j,
        -6.07949930e-01+0.53467919j,   7.44109732e-01-1.06383585j,
        -6.22557065e-01-0.6604577j ,   5.54530896e-01+0.49763768j,
        -1.42655640e+00+1.03849457j,  -8.79197413e-01-0.05384536j,
        -5.91554616e-01-0.29187425j,  -2.09918231e+00+0.65127732j,
        -2.79577518e-01+0.08460458j,  -8.49733033e-01-1.15162916j,
        -5.61159802e-01+0.40323403j,  -1.32856823e-01+0.83145628j,
         1.38730097e-01-0.64695336j,   4.74841673e-01+2.16259344j,
         5.20314820e-02+1.39337638j,   2.40669484e+00+1.44814886j,
         1.02271608e+00-0.64912151j,   6.52379785e-01+0.59394264j,
         2.50946314e-01-1.71467789j,  -3.35061639e-01+1.80721483j,
         1.87006132e+00-0.18721108j,   1.85379814e+00+0.064140

In [8]:
# Run a test
def fun(u):
    f = problem.fieldsPair(m1d,survey)
    f[src,'e_1dSolution'] = u
#     f[src,'b_1d'] = -(m1d.nodalGrad*u)/(1j*simpegem.Utils.EMUtils.omega(src.freq))
    return rx.projectFields(src,m1d,f), lambda t: rx.projectFieldsDeriv(src,m1d,f0,t)
simpeg.Tests.checkDerivative(fun,u0,num=4,plotIt=False)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    5.703e-05     1.929e-06      nan
 1   1.00e-02    5.877e-06     1.954e-08      1.994
 2   1.00e-03    5.894e-07     1.956e-10      1.999
 3   1.00e-04    5.896e-08     1.957e-12      2.000
========================= PASS! =========================
Not just a pretty face Gudni



True

In [9]:
# Test the Jvec derivative.

In [10]:
# print '%s formulation - %s' % (fdemType, comp)
CONDUCTIVITY = 0.01
m0 = np.log(np.ones(problem.mesh.nC)*CONDUCTIVITY)
# mu = np.log(np.ones(problem.mesh.nC)*MU)

if True:
    m0  = m0 + np.random.randn(problem.mesh.nC)*CONDUCTIVITY*1e-1 
#     mu = mu + np.random.randn(prb.mesh.nC)*MU*1e-1

# prb.mu = mu
# survey = prb.survey
def fun(x):
    
    return survey.dpred(m0), lambda x: problem.Jvec(m0, x)
simpeg.Tests.checkDerivative(fun, u0, num=3, plotIt=False)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
Project at freq: 1.000e+02
Project at freq: 1.000e+02


ValueError: dimension mismatch

In [ ]:
problem.getADeriv_m(freq,fields[src,'e_1dSolution'],v)

In [ ]:
%debug

> /home/gudni/anaconda/lib/python2.7/site-packages/scipy/sparse/base.py(327)__mul__()
    326             if other.shape != (N,) and other.shape != (N,1):
--> 327                 raise ValueError('dimension mismatch')
    328 

ipdb> u
> /media/gudni/ExtraDrive1/Codes/python/simpegmt/simpegMT/SurveyMT.py(143)projectFieldsDeriv()
    142                 # bx = Pbx*mkvc(f[src,'b_1d'],2)/mu_0
--> 143                 deriv_complex = Utils.sdiag(1./(Pbx*mkvc(f[src,'b_1d'],2)/mu_0))*(Pex*v) - Utils.sdiag(Pex*mkvc(f[src,'e_1d'],2))*(Utils.sdiag(1./(Pbx*mkvc(f[src,'b_1d'],2)/mu_0)).T*Utils.sdiag(1./(Pbx*mkvc(f[src,'b_1d'],2)/mu_0)))*(Pbx*f._bDeriv_u(src,v)/mu_0)
    144             # elif self.projType is 'Z2D

ipdb> Pex*v
*** ValueError: dimension mismatch
ipdb> v.shape
(180,)


In [ ]:
problem.getA

In [ ]:
dMf_dsig = problem.mesh.getFaceInnerProductDeriv(problem.curModel.sigma)(u0) * problem.curModel.sigmaDeriv
dsig_dm = self.curModel.sigmaDeriv


In [ ]:
problem.mesh.getFaceInnerProductDeriv(problem.curModel.sigma)(u0)

In [ ]:
problem.mesh.getEdgeInnerProductDeriv(problem.curModel.sigma)(u0[1::])